# Import Libraries

In [40]:
import os
import shutil
import time
import json
import requests
import tempfile
import numpy as np
import pandas as pd
from zipfile import ZipFile

import boto3
import sagemaker

from datetime import datetime

# Setup Region, session, and role

In [41]:
region = os.environ["AWS_REGION"]
boto_session = boto3.Session(region_name=region)

account_id = boto_session.client("sts").get_caller_identity()["Account"]

# Get the AWS EventBridge client
event_bridge_client = boto3.client('events')
lambda_client = boto3.client('lambda')

sns_client = boto3.client('sns')
cloudwatch = boto3.client('cloudwatch')

iam_client = boto3.client('iam')

# Initialize variable

In [42]:
# S3 prefix
bucket = "artwork-content-trial-bucket"
prefix="mch-artwork-content"
train_data_dir_prefix="data"
pipeline_dir_prefix="pipeline-data"
# training_input_prefix = "content_ovr_anonymised.csv"
training_input_prefix = "ovr_content_data"

model_alarm_name='Model_Version_Registration_Alarm'
pipeline_alarm_name='SageMaker_Pipeline_Execution_Alarm'
pipeline_name='artwork-content-pipeline-demo'
sns_topic_name = 'ContentPipelineExecutionAlarmTopic'
sns_topic_arn = f'arn:aws:sns:{region}:{account_id}:{sns_topic_name}'
lambda_fcn_name="content-sns-lambda-evt-trigger-fcn"
lambda_arn=f"arn:aws:lambda:us-east-1:791574662255:function:{lambda_fcn_name}"
role_name = "sm-lambda-sns-evt-trigger-role"

# =========================================

# Customized SNS Notification via Lambda function using EventBridge

# ==============================================

# Set SNS Topic

In [45]:
def set_sns_topic(sns_client, sns_topic_name):
    topic_response=None
    topic_arns = sns_client.list_topics()['Topics']
    
    if topic_arns != []:
        for topic in topic_arns:
            if sns_topic_name in topic['TopicArn']:
                print("\n Topic name <<{}>> having SNS ARN <<{}>> already exists. \n".format(sns_topic_name, topic['TopicArn']))
                break
        else:
            print(f'\n New topic <<{sns_topic_name}>> created successfully \n')
            topic_response = sns_client.create_topic(
                                Name=sns_topic_name
                            )
    else:
        print(f'\n New topic <<{sns_topic_name}>> created successfully \n')
        topic_response = sns_client.create_topic(
                            Name=sns_topic_name
                        )
        
    return topic_response

In [46]:
set_sns_topic(sns_client, sns_topic_name)


 Topic name <<ContentPipelineExecutionAlarmTopic>> having SNS ARN <<arn:aws:sns:us-east-1:791574662255:ContentPipelineExecutionAlarmTopic>> already exists. 



# Set SNS Subscribers

In [47]:
# subcribe endpoint to sns
def sns_subscribe(sns_client, sns_topic_arn, protocol, endpoint):
    subscribe_response=None
    
    subscription_arns = sns_client.list_subscriptions_by_topic(TopicArn=sns_topic_arn)['Subscriptions']

    if subscription_arns != []:
        for subscription in subscription_arns:
            if ('Protocol' in subscription) and (subscription['Protocol']==protocol) and (subscription['Endpoint']==endpoint):
                print(f"Subscription name <<{subscription['Endpoint']}>> having protocol <<{subscription['Protocol']}>> already exist.")
                break
        else:
            subscribe_response = sns_client.subscribe(
                                            TopicArn= sns_topic_arn,
                                            Protocol=protocol,
                                            Endpoint=endpoint,
                                            ReturnSubscriptionArn=True
                                        )
            print(f'new subscription - <<{endpoint}>> having protocol <<{protocol}>> created successfully')
    else:
        subscribe_response = sns_client.subscribe(
                                                TopicArn= sns_topic_arn,
                                                Protocol=protocol,
                                                Endpoint=endpoint,
                                                ReturnSubscriptionArn=True
                                            )

        print(f'new subscription - {endpoint} having protocol {protocol} created successfully')
    
    return subscribe_response

# Adding Subscribers

In [48]:

protocol=['email']
endpoint=['vchitrakathi@dminc.com']

for protocol, endpoint in zip(protocol, endpoint):
    sns_subscribe(sns_client, sns_topic_arn, protocol, endpoint)

Subscription name <<vchitrakathi@dminc.com>> having protocol <<email>> already exist.


# Adding lambda permissions

In [49]:
def add_lambda_permissions(lambda_client, lambda_function_name, src_arn, account_id, action=None, prefix=None):
    try :
        response = lambda_client.add_permission(
                FunctionName=lambda_function_name,
                StatementId=f"{prefix}-{int(time.time())}",
                Action=action,
                Principal= f"{prefix}.amazonaws.com",
                SourceArn=src_arn,
                SourceAccount=account_id
            )
        
    except Exception as e:
        print(f"Error occurred while adding permissions: {e}")
        
    return response

def update_lambda_config(lambda_client, lambda_function_name, src_arn):
    try :
        response = lambda_client.update_function_configuration(
                        FunctionName=lambda_function_name,
                        Environment={
                            'Variables': {
                                'SNS_TOPIC_ARN': src_arn
                            }
                        }
                    )
        
    except Exception as e:
        print(f"Error occurred while updating lambda config: {e}")
        
    return response
        

# SNS Lambda trigger to set rule and put lambda function as target

In [50]:
def sns_lambda_trigger(region, account_id, sns_topic_arn, sns_client, event_bridge_client, lambda_client, lambda_function_name, lambda_arn):
    
    response=None

    # Define the EventBridge rule name and description
    rule_name = 'ContentSNSLambdaEventRuleTrigger'
    rule_description = 'Event to trigger Custom SNS Notification via Lambda function on state change of Content Pipeline alarm.'
#     rule_description = 'Trigger sns notification via a Lambda function when a SageMaker pipeline execution is successful and a model is registered'
    
    # Define the Event Pattern
#     event_pattern = {
#           "source": ["aws.sns"],
#           "detail-type": ["AWS API Call via CloudTrail"],
#           "detail": {
#             "eventSource": ["sns.amazonaws.com"]
#           }
#         }
    

    event_pattern = {
                "source": [
                  "aws.cloudwatch"
                ],
                "detail-type": [
                  "CloudWatch Alarm State Change"
                ],
                "detail": {
#                     "alarmName": [
#                         model_alarm_name,
#                         pipeline_alarm_name
#                     ],
                  "state": {
                    "value": [
                      "ALARM", "OK"
                    ]
                  }
                }
              }


#     event_pattern = {
#                   "source": [
#                     "aws.sagemaker"
#                   ],
#                   "detail-type": [
#                     "SageMaker Model Building Pipeline Execution Status Change"
#                   ],
#                   "detail": {
#                     "currentPipelineExecutionStatus": ["Succeeded"]
#                 }
#             }


    # Put the EventBridge rule
    rule_response = event_bridge_client.put_rule(
        Name=rule_name,
        Description=rule_description,
        EventPattern=json.dumps(event_pattern),
        State='ENABLED'
    )
    
    event_rule_arn = rule_response['RuleArn']
    
    # add permissions to Events, SNS
    add_lambda_permissions(lambda_client, lambda_function_name, event_rule_arn, account_id, action='lambda:InvokeFunction', prefix='events')
    add_lambda_permissions(lambda_client, lambda_function_name, sns_topic_arn, account_id, action='lambda:InvokeFunction', prefix='sns')
    update_lambda_config(lambda_client, lambda_function_name, sns_topic_arn)
   
    
    input_msg = {"subject": "Sagemaker Pipeline Execution Results",
                 "message": "Pipeline execution and model registration succeeded"}
    
    # Define the EventBridge target
    lambda_target = {
                'Arn': lambda_arn, # Replace with the actual lambda ARN
                'Id': 'ContentSNSLambdaFunctionTarget',
#                 'Input': json.dumps(input_msg)
            }
    

    # Add the target to the EventBridge rule
    event_bridge_client.put_targets(
        Rule=rule_name,
        Targets=[lambda_target]
    )

    # Wait for the target to become active
    while True:
        response_rule = event_bridge_client.describe_rule(Name=rule_name)
        # response_rule = event_bridge_scheduler.get_schedule(Name=schedule_name)

        if response_rule['State'] == 'ENABLED':
            print('\n === Event Scheduled Successfully ===== \n')
            break
        else:
            print(f'response rule: {response_rule}')
        time.sleep(5)

    # Check the target status
    response = event_bridge_client.list_targets_by_rule(Rule=rule_name)
    
    return response


In [52]:

# response = sns_lambda_trigger(region, account_id, sns_topic_arn, sns_client, event_bridge_client, lambda_client, lambda_fcn_name, lambda_arn)
# print(response)

{'Targets': [{'Id': 'ContentSNSLambdaFunctionTarget', 'Arn': 'arn:aws:lambda:us-east-1:791574662255:function:content-sns-lambda-evt-trigger-fcn'}], 'ResponseMetadata': {'RequestId': '058bf1fb-81f6-41fc-8bd7-1dd9737ed262', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '058bf1fb-81f6-41fc-8bd7-1dd9737ed262', 'content-type': 'application/x-amz-json-1.1', 'content-length': '143', 'date': 'Tue, 28 Feb 2023 09:53:38 GMT'}, 'RetryAttempts': 0}}


# Remove rules

In [ ]:
rule_name = 'ContentSNSLambdaEventRuleTrigger'
event_bridge_client.describe_rule(Name=rule_name)
event_bridge_client.list_targets_by_rule(Rule=rule_name)

In [14]:
rule_name = 'ContentSNSLambdaEventRuleTrigger'
lambda_target_id = 'ContentSNSLambdaFunctionTarget'
# sns_target_id='SNSFunctionTarget'

target_ids=[lambda_target_id] #, sns_target_id]

def remove_event_rule(event_bridge_client, rule_name, target_ids):

    # Disable the EventBridge rule
    event_bridge_client.disable_rule(Name=rule_name)

    # Remove the target for the EventBridge rule
    event_bridge_client.remove_targets(Rule=rule_name, Ids=target_ids)
    
    # delete rule
    event_bridge_client.delete_rule(Name=rule_name)

# =============================================================================
# remove_event_rule(event_bridge_client, rule_name, target_ids)

# ================================================

# Steps related to lambda function and role creation

In [36]:
from lambda_utils import *
from zipfile import ZipFile

In [29]:
#Create IAM role for the Lambda function
# lambda_role = create_role(role_name)

In [31]:
# creating sns policy
# sns_policy_name='lambda-sns-trigger-policy'
# create_sns_policy(role_name, sns_policy_name, region, account_id, lambda_fcn_name, sns_topic_arn)


 The policy lambda-sns-trigger-policy already exists. 



In [32]:
# attach sns policy to the role
# attach_sns_policy(role_name, sns_policy_name)

Attaching <<lambda-sns-trigger-policy>> to <<content-sns-lambda-evt-trigger-role>>
Exception occurred while attaching sns policy- An error occurred (NoSuchEntity) when calling the AttachRolePolicy operation: Policy arn:aws:iam::aws:policy/lambda-sns-trigger-policy does not exist or is not attachable.


In [37]:
lambda_role = {'arn': f"arn:aws:iam::791574662255:role/{role_name}"}
print(lambda_role)

{'arn': 'arn:aws:iam::791574662255:role/sm-lambda-sns-evt-trigger-role'}


In [43]:
#Zip AWS Lambda function code
#Write code to a .py file
# with open('lambda_function.py', 'w') as f:
#     f.write(inspect.cleandoc(lambda_code))


lambda_output_path = 'lambda_output'
module_name='sns_trigger_lambda_function'
fcn_desc = 'AWS Lambda function for automatically triggering SNS Notification after successful pipeline execution.'

os.makedirs(name=lambda_output_path, exist_ok=True)

zip_path = os.path.join(lambda_output_path, 'function.zip')

#Compress file into a zip
with ZipFile(zip_path,'w') as z:
    z.write(f"{module_name}.py")

#Use zipped code as AWS Lambda function code
with open(zip_path, 'rb') as f:
    fcn_code = f.read()

shutil.rmtree(lambda_output_path)

#Create AWS Lambda function
lambda_arn = create_lambda(module_name, lambda_fcn_name, fcn_desc, fcn_code, lambda_role['arn'])

Creating AWS Lambda function ...
SUCCESS: Successfully created AWS Lambda function!


In [44]:
print(lambda_arn)

arn:aws:lambda:us-east-1:791574662255:function:content-sns-lambda-evt-trigger-fcn


In [53]:
# This file contains a small sample of ovr data
sagemaker.s3.S3Uploader.upload("./data/ovr_content_data_v3.csv", 
                               f"s3://{bucket}/{train_data_dir_prefix}")
#wait for file to finish uploading 
time.sleep(5)